In [7]:
# Module Imports
import mariadb
# Execption used to stop the Notebook cell execution politely
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
# Connect to the server and return a Connection object for the db_name
#database.
def connectToDB(db_name):
    try:
        return mariadb.connect(
            user="root",
            password="BLue1998",
            host="172.26.117.187", # Use of localhost because 1/ we don't have
#acces to the docker local network 2/ we have made a port redirection from the
#mariadb server to the host
            port=3306,
            database=db_name
        )
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        raise StopExecution
conn = connectToDB("flight_reservation")

In [8]:
# A bit of dark magic? Just two intrincated query, one to get the TABLES, one
#for each TABLE to get the COLUMNS.
# And a clever use of format to have aligned columns.
def showTablesAndColumns(conn):
    cur = conn.cursor()
    # Query the tables
    cur.execute("SHOW TABLES;")
    # For each table
    for table in cur.fetchall():
        print("**",table[0],"**")
# Defining the output format {:16} indicate at least 16 characters,
#add spaces if necessary
        outputFormat="{:16}\t{:10}\t{}\t{}\t{}\t{}"
# Printing the headers
        print(outputFormat.format("Field","Type","Null","Key","Default","Extra"))
# Query the columns
        cur.execute('SHOW COLUMNS FROM {};'.format(table[0]))
# For each column
        for attribute in cur.fetchall():
# Print the attribute (*attribute give the content of the tuple to
#the format function)
            print(outputFormat.format(*attribute))
showTablesAndColumns(conn)


** Booking **
Field           	Type      	Null	Key	Default	Extra
id_booking      	int(11)   	NO	PRI	None	auto_increment
client_name     	varchar(50)	NO		None	
quantity        	int(2)    	NO		None	
id_flight       	varchar(20)	NO	MUL	None	
** Flight **
Field           	Type      	Null	Key	Default	Extra
id_flight       	varchar(20)	NO	PRI	None	
max_capacity    	int(3)    	NO		None	
current_capacity	int(3)    	NO		None	


In [ ]:
def addFlight(id_flight, maximum_capacity):
    
    pass